# Together_test
M. Lamoureux April 27, 2016

This is my attempt to get drawing working in a Jupyter Notebook. Try drawing on the next panel. (Just click and draw your mouse on the canvas.)





<script src="https://togetherjs.com/togetherjs-min.js"></script>
<a onclick="TogetherJS(this); return false;"><img src="https://togetherjs.com/images/start-togetherjs-blue.png" style="width: 135px" /></a>

<div class="btn-group btn-group-justified" style="margin-right: auto ; margin-top: 10px ; width: 70% "> 
  <a class="btn btn-info color-picker" style="width: 20%">Blue</a>          
  <a class="btn btn-success color-picker" style="width: 20%">Green</a>
  <a class="btn btn-warning color-picker" style="width: 20%">Yellow</a>
  <a class="btn btn-danger color-picker" style="width: 20%">Red</a>
  <a class="btn btn-default color-picker" style="border-bottom-width: 0px; width: 20%">Black</a>
</div>
<div class="clearfix"></div>
<div id="sketchContainer" style="width: 70%; height: 300px; border: 1px solid rgba(0,0,0,0.2)"><canvas id="sketch"></canvas></div>
<div class="btn-group btn-group-justified" style="margin-right: auto; width: 70%;"> 
  <a class="btn btn-info user-color-pick bottom-button" style="width: 30%;">User Color</a>
  <a class="btn btn-success plus-size" style="width: 15%;">
    <i class="fa fa-plus-square"></i>
  </a>
  <a class="btn btn-warning clear" style="width: 15%;">
    <i class="fa fa-times-circle"></i>
  </a>
  <a class="btn btn-danger minus-size" style="width: 15%;">
    <i class="fa fa-minus-square"></i>
  </a>
  <a class="btn btn-default eraser" style="width: 35%; border-top-width: 0px;">
    <i class="fa fa-eraser"></i>
  </a>
</div>



In [1]:
%%javascript
//require(['TogetherJS'], function(TogetherJS){
// get the canvas element and its context
var canvas = document.getElementById('sketch');
var context = canvas.getContext('2d');

// the aspect ratio is always based on 1140x400, height is calculated from width:
canvas.width = $('#sketchContainer').outerWidth();
canvas.height = (canvas.width/1140)*400;
$('#sketchContainer').outerHeight(String(canvas.height) + "px", true);
// scale function needs to know the width/height pre-resizing:
var oWidth = canvas.width;
var oHeight = canvas.height;
var lines = [];

var lastMouse = {
  x: 0,
  y: 0
};

var sendMouse = {
  x: 0,
  y: 0
};

var sendLastMouse = {
  x: 0,
  y: 0
};

// brush settings
context.lineWidth = 2;
context.lineJoin = 'round';
context.lineCap = 'round';
context.strokeStyle = '#000';

// attach the mousedown, mouseout, mousemove, mouseup event listeners.
canvas.addEventListener('mousedown', function (e) {
  lastMouse = {
//    x: e.pageX - this.offsetLeft,
//    y: e.pageY - this.offsetTop
    x: e.pageX - this.getBoundingClientRect().left,
    y: e.pageY - this.getBoundingClientRect().top
  };
  canvas.addEventListener('mousemove', move, false);
}, false);

canvas.addEventListener('mouseout', function () {
  canvas.removeEventListener('mousemove', move, false);
}, false);

canvas.addEventListener('mouseup', function () {
  canvas.removeEventListener('mousemove', move, false);
}, false);

// Sets the brush size:
function setSize(size) {
  context.lineWidth = size;
}

// Sets the brush color:
function setColor(color) {
  context.globalCompositeOperation = 'source-over';
  context.strokeStyle = color;
}

// Sets the brush to erase-mode:
function eraser() {
  context.globalCompositeOperation = 'destination-out';
  context.strokeStyle = 'rgba(0,0,0,1)';
}

// Clears the canvas and the lines-array:
function clear(send) {
  context.clearRect(0, 0, canvas.width, canvas.height);
  lines = [];
//  if (send && TogetherJS.running) {
//    TogetherJS.send({
//      type: 'clear'
//    });
//  }
}

// Redraws the lines from the lines-array:
function reDraw(lines){
  for (var i in lines) {
    draw(lines[i][0], lines[i][1], lines[i][2], lines[i][3], lines[i][4], false);
  }
}
// Draws the lines, called by move and the TogetherJS event listener:
function draw(start, end, color, size, compositeOperation, save) {
  context.save();
  context.lineJoin = 'round'; 
  context.lineCap = 'round';
  // Since the coordinates have been translated to an 1140x400 canvas, the context needs to be scaled before it can be drawn on:
  context.scale(canvas.width/1140,canvas.height/400);
  context.strokeStyle = color;
  context.globalCompositeOperation = compositeOperation;
  context.lineWidth = size;
  context.beginPath();
  context.moveTo(start.x, start.y);
  context.lineTo(end.x, end.y);
  context.closePath();
  context.stroke();
  context.restore();
  if (save) {
    // Won't save if draw() is called from reDraw().
    lines.push([{x: start.x, y: start.y}, {x: end.x, y: end.y}, color, size, compositeOperation]);
  }
}

// Called whenever the mousemove event is fired, calls the draw function:
function move(e) {
  var mouse = {
//    x: e.pageX - this.offsetLeft,
//    y: e.pageY - this.offsetTop
    x: e.pageX - this.getBoundingClientRect().left,
    y: e.pageY - this.getBoundingClientRect().top
  };
  // Translates the coordinates from the local canvas size to 1140x400:
  sendMouse = {
    x: (1140/canvas.width)*mouse.x,
    y: (400/canvas.height)*mouse.y
  };
  sendLastMouse = {
    x: (1140/canvas.width)*lastMouse.x,
    y: (400/canvas.height)*lastMouse.y
  };
  draw(sendLastMouse, sendMouse, context.strokeStyle, context.lineWidth, context.globalCompositeOperation, true);
//  if (TogetherJS.running) {
//    TogetherJS.send({
//      type: 'draw',
//      start: sendLastMouse,
//      end: sendMouse,
//     color: context.strokeStyle,
//     size: context.lineWidth,
//      compositeOperation: context.globalCompositeOperation
//    });
//  }
  lastMouse = mouse;
}

// Listens for draw messages, sends info about the drawn lines:
//TogetherJS.hub.on('draw', function (msg) {
//  if (!msg.sameUrl) {
//      return;
//  }
//  draw(msg.start, msg.end, msg.color, msg.size, msg.compositeOperation, true);
//});


// Clears the canvas whenever someone presses the clear-button
//TogetherJS.hub.on('clear', function (msg) {
//  if (!msg.sameUrl) {
//    return;
//  }
//  clear(false);
//});

// Hello is sent from every newly connected user, this way they will receive what has already been drawn:
//TogetherJS.hub.on('togetherjs.hello', function () {
//  TogetherJS.send({
//    type: 'init',
//    lines: lines
//  });
//});

// Draw initially received drawings:
//TogetherJS.hub.on('init', function (msg) {
//  reDraw(msg.lines);
//  lines = msg.lines;
//});

// JQuery to handle buttons and resizing events, also changes the cursor to a dot resembling the brush size:
$(document).ready(function () {
  // changeMouse creates a temporary invisible canvas that shows the cursor, which is then set as the cursor through css:
  function changeMouse() {
    // Makes sure the cursorSize is scaled:
    var cursorSize = context.lineWidth*(canvas.width/1140); 
    if (cursorSize < 10){
        cursorSize = 10;
    }
    var cursorColor = context.strokeStyle;
    var cursorGenerator = document.createElement('canvas');
    cursorGenerator.width = cursorSize;
    cursorGenerator.height = cursorSize;
    var ctx = cursorGenerator.getContext('2d');

    var centerX = cursorGenerator.width/2;
    var centerY = cursorGenerator.height/2;

    ctx.beginPath();
    ctx.arc(centerX, centerY, (cursorSize/2)-4, 0, 2 * Math.PI, false);
    ctx.lineWidth = 3;
    ctx.strokeStyle = cursorColor;
    ctx.stroke();
    $('#sketch').css('cursor', 'url(' + cursorGenerator.toDataURL('image/png') + ') ' + cursorSize/2 + ' ' + cursorSize/2 + ',crosshair');
  }
  // Init mouse
  changeMouse();

  // Redraws the lines whenever the canvas is resized:
  $(window).resize(function() {
    if ($('#sketchContainer').width() != oWidth) {
      canvas.width = $('#sketchContainer').width();
      canvas.height = (canvas.width/1140)*400;
      $('#sketchContainer').outerHeight(String(canvas.height)+"px", true);
      var ratio = canvas.width/oWidth;
      oWidth = canvas.width;
      oHeight = canvas.height;
      reDraw(lines);
      changeMouse();
    }
  });

  // Clears the canvas:
  $('.clear').click(function () {
    clear(true);
  });

  // Color-button functions:
  $('.color-picker').click(function () {
    var $this = $(this);
    console.log($this);
    setColor($this.css("background-color"));
    changeMouse();
  });

  $('.eraser').click(function () {
    eraser();
    changeMouse();
  });
  // TogetherJS user color:
//  $('.user-color-pick').click(function() {
//    setColor(TogetherJS.require('peers').Self.color);
//    changeMouse();
//  });

  // Increase/decrease brush size:
  $('.plus-size').click(function() {
    setSize(context.lineWidth+3);
    changeMouse();
  });

  $('.minus-size').click(function() {
    if (context.lineWidth > 3) {
      setSize(context.lineWidth-3);
    }
    changeMouse();
  });          
});

//});

<IPython.core.display.Javascript object>

### Some comments

I really like the drawing tool developed by Mozilla, at https://togetherjs.com/examples/drawing/ and wanted to use it as a tool in Jupyter. 

However, it doesn't work, as is, in a Jupyter Notebook. There are four basic problems.

First, Jupyter automatically runs an HTML sanitizer that strips out all the JavaScript. So you need to rewrite the javascript separately, in a %%javascript magic cell. 

Second, in the Jupyter notebook, the relative position of the mouse click has to be computed differently. So, in two places in the js code, I had to change the way the mouse position is computed. Here are the old and new versions:
- OLD VERSION
- x: e.pageX - this.offsetLeft,
- y: e.pageY - this.offsetTop
- NEW, FIXED VERSION
-     x: e.pageX - this.getBoundingClientRect().left,
-     y: e.pageY - this.getBoundingClientRect().top

Third, in the HTML code, you have to be careful about setting up the buttons, otherwise they get all squished to one side. So I just hacked away at the code, until I got something that looked nice.

Fourth, I could not figure out how to load the code in https://togetherjs.com/togetherjs-min.js  So I just deleted all the calls to this code. (This code woudld allow you to draw collaboratively with others on the web. So for now, you cannot collaborate.)

To run the drawing program, you need to run the first cell (to open up the drawing canvas), then run the second cell to "turn on" the drawing commands. In this saved version of the notebook, though, it should run right away. 

ONE LAST PROBLEM: when saved as an HTML file, the drawing works, but scrolling again messes up the alignment of the mouse. 